# Import Modules

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import re
import json
import csv

from src.image_tools import ImageTools
from src.load_image import load_image_to_numpy_array
pd.options.mode.chained_assignment = None  # default='warn'

# CONFIG

In [5]:
"""
NEED TO SET::

MODEL_NAME : Name of model used. For result csv etc.
WORKING_DIR : if images are in a different folder than program
GT : Ground-truth annotations (json format)
PRED : Prediction output (json format)
IMAGE_PATH: path to images from WORKING_DIR

"""
#WORKING_DIR = "C:/Users/Sigurd/OneDriveMS/FYS-3741-MASTER/"
WORKING_DIR = "C:/Users/sigur/OneDriveMS/FYS-3741-MASTER/"
GT = "Labels/Annotations.json"
IMAGE_PATH = WORKING_DIR + "data/data_yoloformat/test/images/"
#RESULT_NAME = "_CONF_TESTING"
#GT = "example_Annotations.json"
#IMAGE_PATH = 'examples/test_im/'

#Faster RCNN 1
#MODEL_NAME = "Faster_RCNN_woDropout"
#MODEL_NAME = "fasterrcnn_wDO_3ksteps"
#PRED = "Labels/fasterrcnn_5ksteps_dropout5_result.json"
#PRED = "Labels/result_faster_rcnn.json"
#RESULT_NAME = "withdropout_confrange_0.1_0.9"
#RESULT_NAME = "_IoUTesting"
#MODEL_NAME = "Faster_RCNN_wdropout3ksteps"

#EfficientDet D1
MODEL_NAME = "EfficientDetD1"
PRED = "Labels/result_efficientDetd1.json"
RESULT_NAME = "_WITH_NMS_CONF01"


#YOLOv4
#MODEL_NAME = "YOLOv4"
#PRED = "Labels/YOLOv4_results.json"
#RESULT_NAME = "_conf_range_0.1_0.9"
#PRED = "Labels/YOLOv4_iou0.5_thresh_0.1_results.json"



# Functions

In [3]:
def json_to_df(json, conf_thresh = None):
    df = pd.read_json(json)

    if conf_thresh is not None:
        pred = df[df['score'] >= conf_thresh]
        return pred
    else:
        return df


class EvaluationTools:
    def __init__(self, im_path, iou_threshold):

        self.im_path = im_path
        self.iou_threshold = iou_threshold
    

    def absolute_to_coordinates(self, bbox):
        """
        convert bbox format from (xmin, ymin, w, h) to (xmin, ymin, xmax, ymax).
        Param: bbox:
                numpy array shape [n,4]
        returns:
                numpy array shape [n,4]
        """
        converted_bbox = bbox.copy()
        converted_bbox[:, 2] = bbox[:, 0] + bbox[:, 2]
        converted_bbox[:, 3] = bbox[:, 1] + bbox[:, 3]

        return converted_bbox


    def get_image_info(self):
        """
        Fetch image height and width
        """
        im = load_image_to_numpy_array(self.im_path)
        im_w = im.shape[1]
        im_h = im.shape[0]
        return im_w, im_h
    
    def get_iou(self, pred, gt):
        """
        Calculate intersection over union (IoU) between all bounding boxes in image.
        Also calculates total pixel area of each bounding box

        param:
            
        returns:
        iou: float(0,1)
        area_bb1: float
        area_bb2: float
        """
        #remove values class/conf values and converte to coordinate form (pascalVOC)
        bb1 = self.absolute_to_coordinates(pred)

        #add extra axis for calculation
        bb1 = bb1[:, None]
        #remove class value and convert to coordinate form (pascalVOC)
        bb2 = gt

        #calculation...
        low = np.s_[...,:2]
        high = np.s_[..., 2:]

        bb1[high] += 1; bb2[high] += 1
        
        #intersect
        intrs = (np.maximum(0,np.minimum(bb1[high],bb2[high])
                            -np.maximum(bb1[low],bb2[low]))).prod(-1)
        #iou
        area_pred = (bb1[high]-bb1[low]).prod(-1)
        area_gt = (bb2[high]-bb2[low]).prod(-1)
        iou = intrs / (area_pred + area_gt -intrs + 1e-16)

        return iou, area_gt
    
    def find_valid_detections(self, iou, iou_threshold, index_list):
        """
        find number TP, FP, FN 
        """

        closest_box = np.max(iou, axis=1)
        closest_gt = np.max(iou, axis=0)

        valid_boxes = closest_box.copy()
        valid_boxes[np.where(closest_box <= iou_threshold)] = 0
        valid_gt = closest_gt.copy()
        valid_gt[np.where(closest_gt <= iou_threshold)] = 0


        tp = np.zeros(len(index_list)); fp = np.zeros(len(index_list)); fn = np.zeros(len(index_list))

        for i in range(len(index_list)):
            try:
                tp[i] += np.count_nonzero(valid_boxes[index_list[i]])
                fp[i] += closest_box[index_list[i]].shape[0] - np.count_nonzero(valid_boxes[index_list[i]])
            except:
                tp[i] += np.count_nonzero(valid_gt[index_list[i]])
                fn[i] = closest_gt[index_list[i]].shape[0] - np.count_nonzero(valid_gt[index_list[i]])
                 
        return tp, fp, fn
        
        
    
    
    def number_of_detections(self, area_gt):

        area = np.ravel(area_gt)
        small_thresh = 32**2
        med_thresh = 96**2
        
        index_full = np.full(area.shape, True)
        index_small = np.full(area.shape, False)
        index_med = np.full(area.shape, False)
        index_large = np.full(area.shape, False)
        
        for i in range(area.shape[0]):
           
            if area[i] < small_thresh:
                index_small[i] = True

            elif area[i] >= small_thresh and area[i] < med_thresh:
                index_med[i] = True

            elif area[i] >= med_thresh:
                index_large[i] = True

            else:
                raise Exception('Value for bbox area is invalid')


        return [index_full, index_small, index_med, index_large]

    def no_predictions(self, gt_bbox):
        """
        Handles situations where there are no valid predictions for a ground truth. Every gt object will be a false negative.
        """

        low = np.s_[...,:2]
        high = np.s_[..., 2:]

        bb2 = gt_bbox
        area_gt = (bb2[high]-bb2[low]).prod(-1)
        indexes = self.number_of_detections(area_gt)

        fn = np.sum(np.array([element*1 for element in indexes]), axis=1)
        
        return fn


def recall(tp, fn):
    return tp / (tp + fn + 1e-10)


def precision(tp, fp):
    return tp / (tp + fp + 1e-10)


def evaluate_results(MODEL_NAME, CONF_THRESH = 0.5, IOU_THRESH = 0.5):
    TP = np.zeros(4)
    FP = np.zeros(4)
    FN = np.zeros(4)
    PRED_LARGE = 0
    PRED_MEDIUM = 0
    PRED_SMALL = 0
    PRED_TOTAL = 0

    pred_df = json_to_df(json = PRED, conf_thresh=CONF_THRESH)
    gt_df = json_to_df(GT)


    for test_index in range(0, len(gt_df['image_id'].unique())):
        #Get current image name
        txt = IM[test_index]
        #Get ID number from image name
        file_number = re.findall(r'\d+', txt)[0]
        #Find all indexes matching current image for GT and pred
        pred = pred_df[pred_df['image_id'] == txt]
        gt = gt_df[gt_df['image_id'] == txt]

        test = EvaluationTools(im_path=IMAGE_PATH+txt, iou_threshold=IOU_THRESH)

        if len(pred) == 0:
            gt_bbox = np.array([val for val in gt['bbox'].to_numpy()])
            
            tp = np.zeros(4)
            fp = np.zeros(4)
            fn = test.no_predictions(gt_bbox)
            TP += tp
            FP += fp
            FN += fn
            continue
        
        #reshape array to remove annoying stuff from bad code
        pred_bbox = np.array([val for val in pred['bbox'].to_numpy()]).round()
        gt_bbox = np.array([val for val in gt['bbox'].to_numpy()])
             

        try:
            iou, area_pred = test.get_iou(pred_bbox, gt_bbox)
            area_indexes = test.number_of_detections(area_pred)
            ########## CHHHHHHEEEEEEEEEEEEEECK TRANSPOSED
            tp, fp, fn = test.find_valid_detections(iou, IOU_THRESH, area_indexes)
            TP += tp
            FP += fp
            FN += fn
        
        except:
            print('Something is wrong with the TP/FP/FN calculation at index {}'.format(IM[test_index]))
        


    

    #col_names = ['Conf_thresh: {}'.format(CONF_THRESH), 'Total', 'small:', 'medium:', 'large:']
    results = {}

    #col_names = ['{}, $\tau = {}$'.format(MODEL_NAME, CONF_THRESH)]

    keys = ['AP@',
            'AR@',
            'Precision Small',
            'Precision Medium',
            'Precision Large',
            'Recall Small',
            'Recall Medium',
            'Recall Large']
    
    #AP/AR
    values = [precision(TP[0], FP[0]), recall(TP[0], FN[0])]
    #precision small/medium/large
    for i in range(1, 4):
        values.append(precision(TP[i], FP[i]))
    #recall small/medium/large
    for i in range(1, 4):
        values.append(recall(TP[i], FN[i]))
    
    #save to dict
    for i in range(len(keys)):
        results[keys[i]] = values[i]
    #print(results)

    #df = pd.DataFrame.from_dict(results, orient = 'index')
    return results, (TP, FP, FN)

# Evaluate model

#Some old code
"""
IM = os.listdir(IMAGE_PATH)
   
    
def main(MODEL_NAME):
    output = pd.DataFrame()
    col_names = []

    # Run on single value
    iou_thresh = 0.5
    #conf_thresh = 0.1
    #print(len(iou_thresh_range))
    
    #results = evaluate_results(MODEL_NAME=MODEL_NAME, CONF_THRESH=conf_thresh, IOU_THRESH=iou_thresh)
    #output = pd.DataFrame([results]).T
    #col_names.append("{}_iou_{}_conf{}".format(MODEL_NAME, iou_thresh, conf_thresh))

    ## For running many tests:

    i = 0
    #iou_thresh_range = range(10, 100, 5)
    conf_thresh_range = range(10, 100, 10)

    for conf in conf_thresh_range:
        conf_thresh = conf / 100

        sys.stdout.write('\r Test: {}, completion: {}%'.format(i, np.round((i + 1) / (len(conf_thresh_range)) * 100, 2)))
        
    
        
        

        results, conf_mat = evaluate_results(MODEL_NAME=MODEL_NAME, CONF_THRESH=conf_thresh, IOU_THRESH=iou_thresh)
        df_dictionary = pd.DataFrame([results]).T
        #print(df_dictionary)
        output = pd.concat([output, df_dictionary], ignore_index=True, axis=1)
        #print(output)
        col_names.append("conf{}".format(MODEL_NAME, conf_thresh))
        #print(col_names)
        i += 1
        sys.stdout.flush()


    output.to_csv(r'results/{}_Evalutation_results.csv'.format(MODEL_NAME+RESULT_NAME), index = True, header=col_names)


main(MODEL_NAME)
"""
#old code done

In [6]:
IM = os.listdir(IMAGE_PATH)


conf_df = pd.DataFrame()
output = pd.DataFrame()
col_names = []
conf_thresh = 0.1

# Run on single value
iou_thresh = 0.5

#print(len(iou_thresh_range))

results, conf_mat = evaluate_results(MODEL_NAME=MODEL_NAME, CONF_THRESH=conf_thresh, IOU_THRESH=iou_thresh)
output = pd.DataFrame([results]).T
col_names.append("{}_iou_{}_conf{}".format(MODEL_NAME, iou_thresh, conf_thresh))

## For running many tests:
"""
i = 0
iou_thresh_range = range(10, 100, 5)
#conf_thresh_range = range(10, 100, 10)

for iou in iou_thresh_range:
    #conf_thresh = conf / 100
    iou_thresh = iou / 100
    sys.stdout.write('\r Test: {}, completion: {}%'.format(i, np.round((i + 1) / (len(iou_thresh_range)) * 100, 2)))
    

    
    

    results, conf_mat = evaluate_results(MODEL_NAME=MODEL_NAME, CONF_THRESH=conf_thresh, IOU_THRESH=iou_thresh)
    df_dictionary = pd.DataFrame([results]).T
    #print(df_dictionary)
    conf = [conf_mat[i][0] for i in range(3)]
    output_conf_mat = pd.DataFrame(conf, columns = [iou])
    conf_df = pd.concat([conf_df, output_conf_mat], ignore_index=True, axis=1)


    output = pd.concat([output, df_dictionary], ignore_index=True, axis=1)
    #print(output)
    col_names.append("conf{}".format(MODEL_NAME, conf_thresh))
    #print(col_names)
    i += 1
    sys.stdout.flush()

"""
#output.to_csv(r'results/{}_Evalutation_results.csv'.format(MODEL_NAME+RESULT_NAME), index = True, header=col_names)
#conf_df.to_csv(r'results/{}_Evalutation_results.csv'.format(MODEL_NAME+RESULT_NAME+'Conf_mat'), index = True, header=col_names)
#print(conf_mat_df)

TP = conf_mat[0]; FP = conf_mat[1]; FN = conf_mat[2]


print('True Positives: [total, small, medium, large]')
print(TP)
print('False Positives: [total, small, medium, large]')
print(FP)
print('False Negatives: [total, small, medium, large]')
print(FN)


#main(MODEL_NAME)


True Positives: [total, small, medium, large]
[3136.    0. 1040. 2096.]
False Positives: [total, small, medium, large]
[275.   0. 119. 156.]
False Negatives: [total, small, medium, large]
[772.   4. 170. 598.]


In [7]:
###PLOT

print('True Positives: [total, small, medium, large]')
print(TP)
print('False Positives: [total, small, medium, large]')
print(FP)
print('False Negatives: [total, small, medium, large]')
print(FN)


r = recall(TP, FN)
p = precision(TP, FP)
print('recall: ')
print(r)
print('precision: ', p)


True Positives: [total, small, medium, large]
[3136.    0. 1040. 2096.]
False Positives: [total, small, medium, large]
[275.   0. 119. 156.]
False Negatives: [total, small, medium, large]
[772.   4. 170. 598.]
recall: 
[0.8024565  0.         0.85950413 0.77802524]
precision:  [0.91937848 0.         0.89732528 0.93072824]


In [ ]:
#mAP = np.mean(ap[8:])
#mAR = np.mean(ar[8:])
#print(mAP)
#print(mAR)


0.4496553479793356
0.6939601151422186


In [ ]:
#print(conf_mat)

(array([12.,  0.,  4.,  8.]), array([3734.,    0., 1073., 2661.]), array([437.,   4., 252., 181.]))
